In [1]:
import sys
import os

SCRIPT_DIR = os.path.dirname(os.path.abspath("..."))
sys.path.append(os.path.dirname(SCRIPT_DIR))
from training.generate import (generate_response, load_model_tokenizer_for_generate, 
                               get_special_token_id, get_special_token_id)
from training.transcript_trainer import PROMPT_FORMAT, create_data_set_from_json_list
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk


/opt/bo_ling/python39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [78]:
file_to_instruction_map = {
    "/opt/home/bo_ling/dataset/michelangelo_so_long.jsonl": "Please answer the following MA helpdesk questions:",
    "/opt/home/bo_ling/dataset/docstrans.jsonl": "Please finish the following doc translation tasks:",
    "/opt/home/bo_ling/dataset/eats_goldenset.jsonl": "Please answer Uber eats products relation questions:"
}
json_file = "/opt/home/bo_ling/dataset/eats_goldenset.jsonl"
dataset_file = json_file.replace(".jsonl", ".hf")
dataset = create_data_set_from_json_list(json_file,
                                         file_to_instruction_map=file_to_instruction_map) 
dataset = dataset.train_test_split(test_size=0.1)
dataset.save_to_disk(dataset_file)

Dataset generator downloaded and prepared to /home/prodadmin/.cache/huggingface/datasets/generator/default-71d75ecb741b2dcb/0.0.0. Subsequent calls will reuse this data.


In [3]:
import json
data = []
max_question_length=2000
max_answer_length=2000
with open("/opt/home/bo_ling/dataset/doc_transcript_pii_data.json") as json_file:
    json_data = json.load(json_file)
for d in json_data:
    try:
        instruction = d['instruction']+":"
        input_text = d["input"][:max_question_length]
        output_text = "\n" + d["output"][:max_answer_length]
        text = PROMPT_FORMAT.format(instruction=instruction,input_text=input_text,output_text=output_text)
        data.append({
                "instruction": instruction,
                "input": input_text, 
                "output": output_text,
                "text": text
            })
    except:
        pass
def gen():
    for d in data:
        yield d
dataset = Dataset.from_generator(gen)
dataset = dataset.train_test_split(test_size=0.1)
dataset.save_to_disk("/opt/home/bo_ling/dataset/doc_transcript_pii_data_simplify.hf")

Dataset generator downloaded and prepared to /home/prodadmin/.cache/huggingface/datasets/generator/default-f24c58b0e2ee275b/0.0.0. Subsequent calls will reuse this data.


In [4]:
new_dataset = load_from_disk("/opt/home/bo_ling/dataset/doc_transcript_pii_data_simplify.hf")
new_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 629797
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 69978
    })
})

In [6]:
new_dataset['train'][1]

{'instruction': 'Extract address from the following input:',
 'input': 'UTAH James J. Brunkan DRIVER LICENSE 4d 154045998 3 DOB 06/03/1966 BRUNKOW 2JAMES JOSEPH 81562 W 1000 N SALT LAKE CITY, UT 84116 5 DD 53339121 9 CLASS D 15 SEX M 16 HGT 6\'-02" 17 WGT 250 lb 18 EYES BLU 19 HAIR BRO I COMMISSIONER OF PUBLIC SAFETY 4a ISS 03/10/2018 4b EXP 06/03/2023 9a END NONE 12 REST B UT USA 06/03/66 DONOR Y',
 'output': '\n1562 W 1000 N',
 'text': 'Extract address from the following input:\nUTAH James J. Brunkan DRIVER LICENSE 4d 154045998 3 DOB 06/03/1966 BRUNKOW 2JAMES JOSEPH 81562 W 1000 N SALT LAKE CITY, UT 84116 5 DD 53339121 9 CLASS D 15 SEX M 16 HGT 6\'-02" 17 WGT 250 lb 18 EYES BLU 19 HAIR BRO I COMMISSIONER OF PUBLIC SAFETY 4a ISS 03/10/2018 4b EXP 06/03/2023 9a END NONE 12 REST B UT USA 06/03/66 DONOR Y\n\n### Response:\n:\n1562 W 1000 N\n'}

In [18]:
parquet_file = "/opt/home/bo_ling/dataset/modeling_data_v1.parquet"
import pandas as pd
df = pd.read_parquet(parquet_file, engine='pyarrow')
count = 0

def gen():
    for index, row in df.iterrows():
        yield {"text": row['context_and_msg']}
dataset = Dataset.from_generator(gen)
dataset = dataset.train_test_split(test_size=0.1)
    
dataset.save_to_disk("/opt/home/bo_ling/dataset/modeling_data_v1.hf")



Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 1796 examples [00:00, 17886.03 examples/s]

Generating train split: 4053 examples [00:00, 20631.90 examples/s]

Generating train split: 6371 examples [00:00, 21793.42 examples/s]

Generating train split: 8686 examples [00:00, 22327.86 examples/s]

Generating train split: 11000 examples [00:00, 22468.39 examples/s]

Generating train split: 13360 examples [00:00, 22846.81 examples/s]

Generating train split: 15714 examples [00:00, 23069.83 examples/s]

Generating train split: 19192 examples [00:00, 23077.28 examples/s]

Generating train split: 21533 examples [00:00, 23166.52 examples/s]

Generating train split: 23885 examples [00:01, 23264.08 examples/s]

Generating train split: 27380 examples [00:01, 23230.48 examples/s]

Generating train split: 29736 examples [00:01, 23316.73 examples/s]

Generating train split: 33173 examples [00:01, 23167.25 examples/s]

Generating train split: 35542 examples [00:01, 

Dataset generator downloaded and prepared to /home/prodadmin/.cache/huggingface/datasets/generator/default-7895c5e842d084b9/0.0.0. Subsequent calls will reuse this data.




Saving the dataset (0/1 shards):   0%|                                                                                                                                                                                      | 0/152615 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):  12%|███████████████████▌                                                                                                                                                  | 18000/152615 [00:00<00:00, 173411.10 examples/s]

Saving the dataset (0/1 shards):  26%|███████████████████████████████████████████▌                                                                                                                          | 40000/152615 [00:00<00:00, 195499.43 examples/s]

Saving the dataset (0/1 shards):  41%|███████████████████████████████████████████████████████████████████▍                                                                                                  | 62000/152615 [00:00<00:0

In [4]:
new_dataset = load_from_disk("/opt/home/bo_ling/dataset/modeling_data_v1.hf")
new_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 152615
    })
    test: Dataset({
        features: ['text'],
        num_rows: 16958
    })
})

In [7]:
import json 
data = []
max_question_length=2000
max_answer_length=2000
with open("/opt/home/bo_ling/dataset/gds_v4.json") as json_file:
    json_data = json.load(json_file)
for d in json_data:
    try:
        instruction = d['instruction'].strip().replace(":", " input:")
        input_text = d["ocr_text"][:max_question_length]
        output_text = "\n" + d["output"][:max_answer_length]
        text = PROMPT_FORMAT.format(instruction=instruction,input_text=input_text,output_text=output_text)
        data.append({
                "instruction": instruction,
                "input": input_text, 
                "output": output_text,
                "text": text
            })
    except:
        pass
def gen():
    for d in data:
        yield d
dataset = Dataset.from_generator(gen)
dataset.save_to_disk("/opt/home/bo_ling/dataset/gds_v4_simplify.hf")

Dataset generator downloaded and prepared to /home/prodadmin/.cache/huggingface/datasets/generator/default-dd0ec69f9326ee46/0.0.0. Subsequent calls will reuse this data.


In [9]:
test_data = load_from_disk("/opt/home/bo_ling/dataset/gds_v4_simplify.hf")
test_data

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 182662
})

In [12]:
count = 0
ins_count = {}
for d in dataset:
    count += 1
    if count >=1000:
        break
    instruction = d['instruction']
    ins_count[instruction] = ins_count.get(instruction, 0) + 1
ins_count   

{'Extract dob from the following input:': 75,
 'Extract issue date from the following input:': 75,
 'Extract expiration date from the following input:': 75,
 'Extract first name from the following input:': 74,
 'Extract middle name from the following input:': 64,
 'Extract last name from the following input:': 74,
 'Extract license class from the following input:': 74,
 'Extract drivers license number from the following input:': 74,
 'Extract zip from the following input:': 74,
 'Extract driving licence issue state from the following input:': 74,
 'Extract address from the following input:': 74,
 'Extract gender from the following input:': 74,
 'Extract city from the following input:': 74,
 'Is the driving license valid for identification?': 44}

In [3]:
parquet_file = "/opt/home/bo_ling/dataset/modeling_data_v1.parquet"
import pandas as pd
df = pd.read_parquet(parquet_file, engine='pyarrow')
from sklearn.model_selection import train_test_split
from training.agent_trainer import generate_data_from_chat
train, test = train_test_split(df, test_size=0.1)
count = 0

def train_gen():
    for index, row in train.iterrows():
        for d in generate_data_from_chat(row['context_and_msg']):
            yield {"text": d}
def test_gen():
    for index, row in test.iterrows():
        for d in generate_data_from_chat(row['context_and_msg']):
            yield {"text": d}
train_dataset = Dataset.from_generator(train_gen)
test_dataset = Dataset.from_generator(test_gen)
dataset = DatasetDict({"train": train_dataset, "test": test_dataset})
    
dataset.save_to_disk("/opt/home/bo_ling/dataset/modeling_data_v1_expanded.hf")


Generating train split: 0 examples [00:00, ? examples/s]
Generating train split: 7909 examples [00:00, 78904.96 examples/s]
Generating train split: 19164 examples [00:00, 98652.93 examples/s]
Generating train split: 36510 examples [00:00, 108104.08 examples/s]
Generating train split: 48000 examples [00:00, 110330.89 examples/s]
Generating train split: 59525 examples [00:00, 111949.49 examples/s]
Generating train split: 71000 examples [00:00, 112801.47 examples/s]
Generating train split: 82521 examples [00:00, 113558.29 examples/s]
Generating train split: 94148 examples [00:00, 114379.95 examples/s]
Generating train split: 105970 examples [00:00, 115541.35 examples/s]
Generating train split: 117675 examples [00:01, 116000.65 examples/s]
Generating train split: 135072 examples [00:01, 115967.78 examples/s]
Generating train split: 152210 examples [00:01, 115337.20 examples/s]
Generating train split: 163937 examples [00:01, 115795.80 examples/s]
Generating train split: 181292 examples [00

Generating train split: 1600766 examples [00:13, 117178.46 examples/s]
Generating train split: 1618152 examples [00:14, 116753.17 examples/s]
Generating train split: 1630000 examples [00:14, 116928.28 examples/s]
Generating train split: 1641778 examples [00:14, 117133.87 examples/s]
Generating train split: 1658143 examples [00:14, 113817.48 examples/s]
                                                                      

Dataset generator downloaded and prepared to /home/prodadmin/.cache/huggingface/datasets/generator/default-ead54ea4939cc760/0.0.0. Subsequent calls will reuse this data.



Generating train split: 0 examples [00:00, ? examples/s]
Generating train split: 11000 examples [00:00, 108369.26 examples/s]
Generating train split: 22683 examples [00:00, 113255.44 examples/s]
Generating train split: 34056 examples [00:00, 113466.96 examples/s]
Generating train split: 45769 examples [00:00, 114843.03 examples/s]
Generating train split: 57403 examples [00:00, 115344.20 examples/s]
Generating train split: 69000 examples [00:00, 115483.80 examples/s]
Generating train split: 80800 examples [00:00, 116289.92 examples/s]
Generating train split: 98010 examples [00:00, 115634.69 examples/s]
Generating train split: 115370 examples [00:01, 115667.31 examples/s]
Generating train split: 126977 examples [00:01, 115643.15 examples/s]
Generating train split: 138764 examples [00:01, 116223.13 examples/s]
Generating train split: 150516 examples [00:01, 116584.72 examples/s]
Generating train split: 168000 examples [00:01, 116325.34 examples/s]
Generating train split: 184734 examples 

Dataset generator downloaded and prepared to /home/prodadmin/.cache/huggingface/datasets/generator/default-46880ca7efe99230/0.0.0. Subsequent calls will reuse this data.



Saving the dataset (1/3 shards):  33%|██████████████████████████████████████████████████████▎                                                                                                            | 552715/1658143 [00:00<00:00, 1665159.49 examples/s]
                                                                                                                                                                                                                                                              
Saving the dataset (1/1 shards): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 184734/184734 [00:00<00:00, 1792916.48 examples/s]
                                                                                                                                                                                                                                          

In [4]:
new_dataset = load_from_disk("/opt/home/bo_ling/dataset/modeling_data_v1_expanded.hf")
new_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1658143
    })
    test: Dataset({
        features: ['text'],
        num_rows: 184734
    })
})

In [7]:
new_dataset['train'][100]

{'text': 'Specific Information: The restaurant name is Epic Wings- Vista and the order value is $58.6. The estimated delivery time is 20:10 and latest arrival time is 20:20. Order status is 4.\n\nAngelica: Hi aakash I been trying to contact my delivery person that has been in the same spot for idk how long already \n\nAngelica: No response but reading my messages \n\nAngelica: And I asked for priority \n\nAngelica: While shes been in the same spot near where she should be picking up my food \n\nAngelica: I want to know if theres a problem \n\nAngelica: I want her to cancel my pickup if she cant fulfill what Im lying for \n\nAngelica: *paying \n\nAngelica: Taking way longer than it should \n\nAngelica: Ok \n\nAngelica: I just wished the lady responded she read my message \nagent: Are we still connected? <|endofsentence|>'}